In [32]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd

In [6]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

In [7]:
pt_dataset = pt.get_dataset('irds:reneuir-2024/dl-top-1000-docs-20240701-training')

In [10]:
plaid_mono_t5 = tira.get_run_output('reneuir-2024/reneuir-baselines/plaid-mono-t5', 'reneuir-2024/dl-top-1000-docs-20240701-training')
plaid_mono_t5 = pt.transformer.get_transformer(pt.io.read_results(plaid_mono_t5 + '/run.txt'))

bm25_mono_t5 = tira.get_run_output('reneuir-2024/tira-ir-starter/MonoT5 3b (tira-ir-starter-gygaggle)', 'reneuir-2024/dl-top-1000-docs-20240701-training')
bm25_mono_t5 = pt.transformer.get_transformer(pt.io.read_results(bm25_mono_t5 + '/run.txt'))

In [20]:
plaid_mono_t5(pt_dataset.get_topics('title'))

,qid,query,docno,rank,score,name
0,1030303,who is aziz hashim,8726437,1,-0.000837,castorini/monot5-3b-msmarco
1,1030303,who is aziz hashim,8726435,2,-0.001651,castorini/monot5-3b-msmarco
2,1030303,who is aziz hashim,7156982,3,-0.002680,castorini/monot5-3b-msmarco
3,1030303,who is aziz hashim,8726436,4,-0.003766,castorini/monot5-3b-msmarco
4,1030303,who is aziz hashim,8726432,5,-0.005998,castorini/monot5-3b-msmarco
...,...,...,...,...,...,...
96995,146187,difference between a mcdouble and a double che...,8555156,996,-19.700455,castorini/monot5-3b-msmarco
96996,146187,difference between a mcdouble and a double che...,7161804,997,-19.804131,castorini/monot5-3b-msmarco
96997,146187,difference between a mcdouble and a double che...,1798170,998,-19.822765,castorini/monot5-3b-msmarco
96998,146187,difference between a mcdouble and a double che...,2458301,999,-20.032055,castorini/monot5-3b-msmarco


In [33]:
results_concatenated = pd.concat([plaid_mono_t5(pt_dataset.get_topics('title')), bm25_mono_t5(pt_dataset.get_topics('title'))])

results = {}

for _, i in results_concatenated.iterrows():
    qid = str(i['qid'])
    docno = str(i['docno'])
    if qid not in results:
        results[qid] = {}
    if docno not in results[qid]:
        results[qid][docno] = -100

        results[qid][docno] += i['score']

results_pt = []

for qid, docs in results.items():
    for docno, score in docs.items():
        results_pt.append({'qid': str(qid), 'docno': str(docno), 'score': score})
results_pt = pd.DataFrame(results_pt)
persist_and_normalize_run(results_pt, 'fusion-monot5', '.')

The run file is normalized outside the TIRA sandbox, I will store it at ".".
Done. run file is stored under "./run.txt".
